# Building a Debugger with Cassette
![](https://avatars1.githubusercontent.com/u/46803805?s=156&v=4)
<img src="https://raw.githubusercontent.com/jrevels/Cassette.jl/master/docs/img/cassette-logo.png" alt="drawing" width="256"/>
## Lyndon White (@oxinabox)
### RSE at Invenia Labs

## With Thanks
 - Tim Holy
 - Jarrett Revels
 - Keno Fischer
 - Kristoffer Carlsson

## Outline:
 - About MagneticReadHead 
 - 3 Debuggers:
    - Overdubbing Calls only (MRH v0.1)
    - Track everything (MRH v0.2)
    - Track nothing (MRH v0.3)
    

### What is Magnetic Read Head ?

A Magentic Read Head sits above a cassette tape (or a magnetic disk), and reads the content off of it.

 - The MRH is a debugger that works by modifying the IR level code during the compilation process, to include debug instrumentation.
 - it is completely compiled using the standard julia compiler.
 - Incontrast to Debugger.jl which uses JuliaInterpreter.jl
 - Incontrast to Gallium which used some kind of LLVM magic

### Performance:
```julia
function summer(A)
   s = zero(eltype(A))
   for a in A
       s += a
   end
   return s
end
```

**x**|**summer(rand(10))**|**summer(rand(100))**|**summer(rand(1000))**
-----|-----|-----|-----
Debugger|8E+05|1E+05|4E+04
MRH|1E+05|6E+04|4E+04
Native|1E+00|1E+00|1E+00

## Performance of debuggers is complicated:
 - MRH always has orders of magnitude less allocations than Debugger.jl
    -  But this does not always translate into speed, so is not that interesting.
    -  Native for this code does not allocate at all.
 - MRH is very slow on first run as it has to recompile every method it touchs,
 and first run-only run is common when debugging.
 - This kind of instrumentation destroys SIMD, and potentially breaks CPU pipelining.


## Insane performance blackholes exist

```
julia> foo() = Complex.(rand(1,2), rand(1,2)) * rand(Int, 2,1);

julia> @btime foo();
  297.770 ns (9 allocations: 720 bytes)

julia> @btime Debugger.@run foo();
  15.472 ms (46982 allocations: 1.78 MiB)

julia> @time MagneticReadHead.@run foo()
  <Hangs for over 30 minutes>
```

## How Does Cassette Work?
 - It is not magic, Cassette is not specially baked into the compiler.
 - `@generated` function can return a `Expr` **or** a `CodeInfo`
 - We return a `CodeInfo` based on a modified version of one for a function argument. Its a bit like a macro with dynamic scope.
 - This capacity allows on to build AD tools, Mocking tools, Debuggers and more.
     - Contrast: Swift for TensorFlow which is adding AD into the compiler.

### Manual pass
```julia
call_and_print(f, args...) = (println(f, " ", args); f(args...))

@generated function rewritten(f)
    meth = first(methods(f.instance, Tuple{}))
    ci = copy(Base.uncompressed_ast(meth))
    for ii in 1:length(ci.code)
        if ci.code[ii] isa Expr && ci.code[ii].head==:call
            func = GlobalRef(Main, :call_and_print)
            ci.code[ii] = Expr(:call, func, statement.args...)
        end
    end
    return ci
end
```

### Result of our manual pass:
```julia
julia> foo() = 2*(1+1);
julia> rewritten(foo)
+ (1, 1)
* (2, 2)
4
```

# What is next for MagneticReadHead ? 
 - Bug squashing
 - Rewrite in IRTools.jl
 - Think about meta-debugging instrumentation.
 - More alignment with Debugger.jl
 - /Apply this kind of tooling into Debugger.jl

 